#### Libraries/Imports

In [52]:
import requests

from bson import ObjectId
from pydantic.json import ENCODERS_BY_TYPE

from pydantic import BaseModel, Field
from typing import List, Optional

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler

##### Helper Models

In [53]:
class PydanticObjectId(ObjectId):
    """
    Object Id field. Compatible with Pydantic.
    """

    @classmethod
    def __get_validators__(cls):
        yield cls.validate

    @classmethod
    def validate(cls, v):
        return PydanticObjectId(v)

    @classmethod
    def __modify_schema__(cls, field_schema: dict):
        field_schema.update(
            type="string",
        )


ENCODERS_BY_TYPE[PydanticObjectId] = str

In [54]:
class Product(BaseModel):
    id: Optional[PydanticObjectId] = Field(None, alias="_id")
    name: str
    brand: str
    allergens: List[str]
    ingredients: List[str]
    calories: float
    fat: float
    carbohydrates: float
    protein: float
    rating: int

##### API URL

In [83]:
url = 'http://localhost:5000/'

In [84]:
try:
    response = requests.get(url)

    if response.status_code == 200:
        print("Response content:")
        response_content = response.json()
        print(response_content)
    else:
        print(f"Failed to retrieve data. Status code: {response.status_code}")

except requests.exceptions.RequestException as e:
    print(f"Error occurred: {e}")

Error occurred: HTTPConnectionPool(host='localhost', port=5000): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f7f93686110>: Failed to establish a new connection: [Errno 111] Connection refused'))


In [71]:
all_products = [Product(**product_dict) for product_dict in response_content.get("products", [])]

# user_basket = all_products[:3] + all_products[-2:]
#user_basket = all_products[-4:]
user_basket = []
#user_allergens = ['en:milk']
user_allergens = []

print("User basket:")
for product in user_basket:
    print(product)

print("Available products:")
for product in available_products:
    print(product)

User basket:
Available products:
id=ObjectId('65d25943c2ef35ebebb785e4') name='Paine boiereasca neagra' brand='Velrom' allergens=['en:gluten', 'en:milk'] ingredients=[] calories=250.0 fat=2.4 carbohydrates=39.0 protein=9.6 rating=4
id=ObjectId('65e0ac46d83183b40dcf2994') name='Paine Alba' brand='Pangroup' allergens=[] ingredients=[] calories=245.0 fat=1.0 carbohydrates=52.0 protein=7.5 rating=0
id=ObjectId('65e0ad3bd83183b40dcf2995') name='Pate Porc Picant' brand='Ardealul' allergens=[] ingredients=[] calories=196.0 fat=16.0 carbohydrates=4.0 protein=9.0 rating=2
id=ObjectId('65e0b1aad8318366c3f58bbf') name='Smart Bucket' brand='KFC' allergens=[] ingredients=[] calories=197.0 fat=9.13 carbohydrates=19.02 protein=9.09 rating=0
id=ObjectId('65e0fe2dd83183aa03972937') name='Viva Chips Chicken' brand='European Food International' allergens=[] ingredients=[] calories=549.0 fat=33.0 carbohydrates=57.0 protein=5.0 rating=1
id=ObjectId('65e1e0a3d831834c3c639198') name='Product With Allergens' 

##### Compute Product Similarity

In [73]:
def compute_similarity(product1, product2, user_allergens = []):
    scaler = MinMaxScaler()
    attributes1 = scaler.fit_transform([[product1.calories, product1.fat, product1.carbohydrates, product1.protein]])
    attributes2 = scaler.transform([[product2.calories, product2.fat, product2.carbohydrates, product2.protein]])

    numeric_similarity = cosine_similarity(attributes1, attributes2)[0][0]

    # Jaccard
    ingredients_set1 = set(product1.ingredients)
    ingredients_set2 = set(product2.ingredients)
    
    if len(ingredients_set1) == 0 and len(ingredients_set2) == 0:
        ingredients_similarity = 1.0
    elif len(ingredients_set1) == 0 or len(ingredients_set2) == 0:
        ingredients_similarity = 0.0
    else:
        ingredients_similarity = len(ingredients_set1.intersection(ingredients_set2)) / len(ingredients_set1.union(ingredients_set2))

    allergen_penalty = 0.0
    if any(allergen in product2.allergens for allergen in user_allergens):
        allergen_penalty = 0.035


    similarity_score = 0.7 * numeric_similarity + 0.3 * ingredients_similarity - allergen_penalty
    return similarity_score


def recommend_products(user_basket, all_products, user_allergens = [], N = 9):
    if not user_basket:
        return sorted(all_products, key=lambda x: x.rating, reverse=True)
    
    available_products = [product for product in all_products if product not in user_basket]
    recommendations = []

    for available_product in available_products:
        similarity_scores = [compute_similarity(user_product, available_product, user_allergens) for user_product in user_basket]
        average_similarity = sum(similarity_scores) / len(similarity_scores)
        
        print(f"Product: {available_product.name}")
        print(f"Similarity Scores: {similarity_scores}")
        print(f"Average Similarity: {average_similarity}")
        
        recommendations.append((available_product, average_similarity))

    recommendations.sort(key=lambda x: x[1], reverse=True)
    recommended_products = [product for product, _ in recommendations]
    return recommended_products[:N]

##### Test Implementation

In [74]:
recommendations_count = len(all_products)

top_recommendations = recommend_products(user_basket, all_products, user_allergens, recommendations_count)

print(f"Top {recommendations_count} recommended products:")
for recommendation in top_recommendations:
    print(recommendation)

print("Available products:")
for product in all_products:
    print(f"product")

Top 32 recommended products:
id=ObjectId('65d25943c2ef35ebebb785e4') name='Paine boiereasca neagra' brand='Velrom' allergens=['en:gluten', 'en:milk'] ingredients=[] calories=250.0 fat=2.4 carbohydrates=39.0 protein=9.6 rating=4
id=ObjectId('65e0ad3bd83183b40dcf2995') name='Pate Porc Picant' brand='Ardealul' allergens=[] ingredients=[] calories=196.0 fat=16.0 carbohydrates=4.0 protein=9.0 rating=2
id=ObjectId('65e0fe2dd83183aa03972937') name='Viva Chips Chicken' brand='European Food International' allergens=[] ingredients=[] calories=549.0 fat=33.0 carbohydrates=57.0 protein=5.0 rating=1
id=ObjectId('65e1e85ad831834c3c63919b') name='Pastile de codat' brand='PAITON BLYAD' allergens=['en:milk', 'en:sesame-seeds', 'en:crustaceans', 'en:fish', 'en:gluten'] ingredients=[] calories=1.0 fat=0.0 carbohydrates=0.0 protein=0.0 rating=1
id=ObjectId('65ea411ff69a9c0e5cb45921') name='Merci Finest Selection' brand='Storck' allergens=['en:soybeans', 'en:eggs', 'en:nuts', 'en:peanuts', 'en:milk', 'en:p